In [4]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

In [3]:
fm = matplotlib.font_manager.fontManager
fm.addfont("./仿宋_GB2312.TTF")
fm.addfont("./times.ttf")

# 设置中文字体和负号正常显示,启用LaTeX排版
plt.rcParams["font.sans-serif"] = ["FangSong_GB2312", "times"]
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
sic_10 = pd.read_excel("../data/附件/附件1.xlsx")
sic_15
si_10
si_15